In [543]:
import pandas as pd
import numpy as np

In [544]:
path_leopard = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\дальневосточный леопард\objects.csv"
path_tiger = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Тигры\objects.csv"
path_princess = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Принцесса_400\objects.csv"

In [545]:
df_leopard = pd.read_csv(path_leopard)
df_tiger = pd.read_csv(path_tiger)
df_princess = pd.read_csv(path_princess)

In [546]:
def len_set(df):
    list_all_image_name = df['id']
    return len(list_all_image_name), len(set(list_all_image_name))

# Изображения, которые фильтранулись надо бы "изолировать в отдельную папку"

In [547]:
# from collections import Counter
# def ununique_el(df):
#     '''Функция находит неуникальные названия картинок, кроме того находит картинки, которые не описаны в датафрейме
#     Это именно косяк датафрейма, в нем есть строки с повторяющимися id, а так же есть неописанные в датафрейме изображения
#     К счастью их немного)'''
#     list_images = list(df['id'])
#     ununique_el = [k for k, v in Counter(list_images).items() if v > 1]
#     return ununique_el

In [548]:
from collections import Counter
import os
def filter_nonrelevant_images(df, directory):
    '''Функция находит неуникальные названия картинок, кроме того находит картинки, которые не описаны в датафрейме
    Это именно косяк датафрейма, в нем есть строки с повторяющимися id, а так же есть неописанные в датафрейме изображения
    К счастью их немного)'''
    
    '''Работа с датафреймом, описывающим картинки'''
    list_images = list(df['id']) 
    ununique_el = [k for k, v in Counter(list_images).items() if v > 1] #id, которые в датафрейме повторяются - эти строки просто удаляем из датафрейма
    
    '''Работа с папкой с картинками'''
    files = os.listdir(directory) #все файлы в папке
    images = set(list(filter(lambda x: x.endswith('.jpg'), files))) #отфильтрованные как jpg
    
    df_images = set(df['id']) #- set(ununique_el)  #Изображения, описанные в датафрейме
    difference = images - df_images   #изображения, не описанные в датафрейме - их фильтруем в отдельную папку
    nonrelevant_images = list(set.union(set(ununique_el), difference)) 
    
    print('Пересечение множеств ', len(set.intersection(set(difference), set(ununique_el)))) # длина пересечения равна длине ununique_el, то есть ununique_el включает полностью difference
    #поэтому просто достаточно удалить ununique_el и из датафрейма и из изображений
    print('Длины множеств' ,len(ununique_el), len(difference))
    
    print('длина до удаления лишних строк', df.shape[0])
    df = df[~df['id'].isin(ununique_el)]
    df.reset_index(inplace = True)
    print('длина после удаления лишних строк', df.shape[0])
    
    return df, ununique_el + list(difference)

In [549]:
import shutil
def filter_images(ununique_el, directory, cat):
    if cat == "Леопард":
        path_filter = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Отфильтрованные изображения\Леопарды"
    elif cat == "Тигр":
        path_filter = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Отфильтрованные изображения\Тигры"
    elif cat == "Принцeссa":
        path_filter = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Отфильтрованные изображения\Принцесса"
        
    for name_image in ununique_el:
        shutil.move(directory + f"\\" + name_image , path_filter)

In [550]:
directory_leopard =  r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\дальневосточный леопард"
df_leopard, ununique_el = filter_nonrelevant_images(df_leopard, directory_leopard)
filter_images(ununique_el, directory_leopard, "Леопард")

Пересечение множеств  0
Длины множеств 58 41
длина до удаления лишних строк 3517
длина после удаления лишних строк 3401


In [551]:
directory_tiger =  r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Тигры"
df_tiger, ununique_el = filter_nonrelevant_images(df_tiger, directory_tiger)
filter_images(ununique_el, directory_tiger, "Тигр")

Пересечение множеств  0
Длины множеств 36 5
длина до удаления лишних строк 3537
длина после удаления лишних строк 3459


In [552]:
directory_princess =  r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Принцесса_400"
df_princess, ununique_el = filter_nonrelevant_images(df_princess, directory_princess)
filter_images(ununique_el, directory_princess, "Принцeссa")

Пересечение множеств  0
Длины множеств 1 13
длина до удаления лишних строк 361
длина после удаления лишних строк 359


In [553]:
def coordinates(df):
    bbox = df['bbox'].apply(lambda x: x.split(' ')) 
    bbox = np.array([i for i in bbox])
    bbox = bbox.astype('float64') 
    
    df['xmin'] = bbox[:, 0]
    df['ymin'] = bbox[:, 1]
    df['xmax'] = bbox[:, 2]
    df['ymax'] = bbox[:, 3]
    return df

In [554]:
df_leopard = coordinates(df_leopard)
df_leopard

,index,id,center,class,subclass,bbox,convex,headtail,orient,relevance,mark_tool,xmin,ymin,xmax,ymax
0,0,01020010_S02.jpg,1849.50 1317.50,Leopard,NaN,1218.00 1000.00 2481.00 1635.00,NaN,NaN,NaN,0,BBox,1218.0,1000.0,2481.0,1635.0
1,1,212_IMG_0841_S02.jpg,1112.00 1118.00,Leopard,NaN,554.00 730.00 1670.00 1506.00,NaN,NaN,NaN,0,BBox,554.0,730.0,1670.0,1506.0
2,2,212_IMG_0840_S02.jpg,1502.50 1059.00,Leopard,NaN,997.00 724.00 2008.00 1394.00,NaN,NaN,NaN,0,BBox,997.0,724.0,2008.0,1394.0
3,3,212_IMG_0839_S02.jpg,1734.50 1048.50,Leopard,NaN,1424.00 734.00 2045.00 1363.00,NaN,NaN,NaN,0,BBox,1424.0,734.0,2045.0,1363.0
4,4,212_IMG_0838_S02.jpg,1934.50 1028.00,Leopard,NaN,1824.00 763.00 2045.00 1293.00,NaN,NaN,NaN,0,BBox,1824.0,763.0,2045.0,1293.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3396,3511,216_08210217_S02.jpg,1093.50 1060.50,Leopard,NaN,568.00 756.00 1619.00 1365.00,NaN,NaN,NaN,0,BBox,568.0,756.0,1619.0,1365.0
3397,3512,236_IMG_3658_S02.jpg,1215.00 964.50,Leopard,NaN,406.00 436.00 2024.00 1493.00,NaN,NaN,NaN,0,BBox,406.0,436.0,2024.0,1493.0
3398,3513,246_IMG_1651_S02.jpg,241.50 954.50,Leopard,NaN,1.00 739.00 482.00 1170.00,NaN,NaN,NaN,0,BBox,1.0,739.0,482.0,1170.0
3399,3514,270_PICT1301_S02.jpg,2146.00 1285.00,Leopard,NaN,1704.00 691.00 2588.00 1879.00,NaN,NaN,NaN,0,BBox,1704.0,691.0,2588.0,1879.0


In [555]:
df_tiger = coordinates(df_tiger)
df_tiger

,index,id,center,class,subclass,bbox,convex,headtail,orient,relevance,mark_tool,xmin,ymin,xmax,ymax
0,0,001_1.jpg,899.50 755.00,Tiger,NaN,148.00 406.00 1651.00 1104.00,NaN,NaN,NaN,0,BBox,148.0,406.0,1651.0,1104.0
1,1,002_758.jpg,1340.00 830.00,Tiger,NaN,730.00 524.00 1950.00 1136.00,NaN,NaN,NaN,0,BBox,730.0,524.0,1950.0,1136.0
2,2,002_76.jpg,2593.50 1531.50,Tiger,NaN,1928.00 850.00 3259.00 2213.00,NaN,NaN,NaN,0,BBox,1928.0,850.0,3259.0,2213.0
3,3,002_761.jpg,324.50 799.00,Tiger,NaN,1.00 582.00 648.00 1016.00,NaN,NaN,NaN,0,BBox,1.0,582.0,648.0,1016.0
4,4,002_762.jpg,570.50 779.50,Tiger,NaN,138.00 564.00 1003.00 995.00,NaN,NaN,NaN,0,BBox,138.0,564.0,1003.0,995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,3532,109_IMG_0158_S02.jpg,472.50 765.00,Tiger,NaN,0.00 512.00 945.00 1018.00,NaN,NaN,NaN,0,BBox,0.0,512.0,945.0,1018.0
3455,3533,109_IMG_0157_S02.jpg,599.00 749.00,Tiger,NaN,94.00 495.00 1104.00 1003.00,NaN,NaN,NaN,0,BBox,94.0,495.0,1104.0,1003.0
3456,3534,109_IMG_0154_S02.jpg,916.00 730.00,Tiger,NaN,442.00 493.00 1390.00 967.00,NaN,NaN,NaN,0,BBox,442.0,493.0,1390.0,967.0
3457,3535,109_IMG_0143_S02.jpg,1683.50 705.50,Tiger,NaN,1337.00 530.00 2030.00 881.00,NaN,NaN,NaN,0,BBox,1337.0,530.0,2030.0,881.0


In [556]:
df_princess = coordinates(df_princess)
df_princess

,index,id,center,class,subclass,bbox,convex,headtail,orient,relevance,mark_tool,xmin,ymin,xmax,ymax
0,0,Princess_387.jpg,2126.50 1212.00,Tiger,NaN,989.00 630.00 3264.00 1794.00,NaN,NaN,NaN,0,BBox,989.00,630.0,3264.00,1794.0
1,1,Princess_388.jpg,802.00 1678.00,Tiger,NaN,0.00 1000.00 1604.00 2356.00,NaN,NaN,NaN,0,BBox,0.00,1000.0,1604.00,2356.0
2,2,Princess_389.jpg,1347.36 1694.50,Tiger,NaN,-3.29 981.00 2698.00 2408.00,NaN,NaN,NaN,0,BBox,-3.29,981.0,2698.00,2408.0
3,3,Princess_39.jpg,387.50 985.00,Tiger,NaN,0.00 758.00 775.00 1212.00,NaN,NaN,NaN,0,BBox,0.00,758.0,775.00,1212.0
4,4,Princess_391.jpg,2479.50 1219.50,Tiger,NaN,1698.00 640.00 3261.00 1799.00,NaN,NaN,NaN,0,BBox,1698.00,640.0,3261.00,1799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,356,Princess_107.jpg,2532.00 1315.50,Tiger,NaN,1800.00 456.00 3264.00 2175.00,NaN,NaN,NaN,0,BBox,1800.00,456.0,3264.00,2175.0
355,357,Princess_1.jpg,740.29 500.00,Tiger,NaN,206.00 156.00 1274.58 844.00,NaN,NaN,NaN,0,BBox,206.00,156.0,1274.58,844.0
356,358,Princess_240.jpg,2830.00 1536.00,Tiger,NaN,2396.00 725.00 3264.00 2347.00,NaN,NaN,NaN,0,BBox,2396.00,725.0,3264.00,2347.0
357,359,Princess_99.jpg,2063.50 1163.00,Tiger,NaN,863.00 621.00 3264.00 1705.00,NaN,NaN,NaN,0,BBox,863.00,621.0,3264.00,1705.0


In [557]:
from PIL import Image

def determ_width_height(df, cat):
    if cat == "Леопард":
        path = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\дальневосточный леопард"
    elif cat == "Тигр":
        path = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Тигры"
    elif cat == "Принцeссa":
        path = r"C:\Users\petili\Desktop\Хакатоны\Самара\Датасет\Минприроды\Train\Принцесса_400"
    
    width = []
    height = []
    
    for image in df['id']:
        name = path +  f"\\" + image
        im = Image.open(name)
        (w, h) = im.size
        width.append(w)
        height.append(h)
        
    df['width'] = width
    df['height'] = height
        
    return df

In [558]:
%%time
df_leopard = determ_width_height(df_leopard, "Леопард")
df_leopard

Wall time: 838 ms


,index,id,center,class,subclass,bbox,convex,headtail,orient,relevance,mark_tool,xmin,ymin,xmax,ymax,width,height
0,0,01020010_S02.jpg,1849.50 1317.50,Leopard,NaN,1218.00 1000.00 2481.00 1635.00,NaN,NaN,NaN,0,BBox,1218.0,1000.0,2481.0,1635.0,3264,2448
1,1,212_IMG_0841_S02.jpg,1112.00 1118.00,Leopard,NaN,554.00 730.00 1670.00 1506.00,NaN,NaN,NaN,0,BBox,554.0,730.0,1670.0,1506.0,2048,1536
2,2,212_IMG_0840_S02.jpg,1502.50 1059.00,Leopard,NaN,997.00 724.00 2008.00 1394.00,NaN,NaN,NaN,0,BBox,997.0,724.0,2008.0,1394.0,2048,1536
3,3,212_IMG_0839_S02.jpg,1734.50 1048.50,Leopard,NaN,1424.00 734.00 2045.00 1363.00,NaN,NaN,NaN,0,BBox,1424.0,734.0,2045.0,1363.0,2048,1536
4,4,212_IMG_0838_S02.jpg,1934.50 1028.00,Leopard,NaN,1824.00 763.00 2045.00 1293.00,NaN,NaN,NaN,0,BBox,1824.0,763.0,2045.0,1293.0,2048,1536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3396,3511,216_08210217_S02.jpg,1093.50 1060.50,Leopard,NaN,568.00 756.00 1619.00 1365.00,NaN,NaN,NaN,0,BBox,568.0,756.0,1619.0,1365.0,2048,1536
3397,3512,236_IMG_3658_S02.jpg,1215.00 964.50,Leopard,NaN,406.00 436.00 2024.00 1493.00,NaN,NaN,NaN,0,BBox,406.0,436.0,2024.0,1493.0,2048,1536
3398,3513,246_IMG_1651_S02.jpg,241.50 954.50,Leopard,NaN,1.00 739.00 482.00 1170.00,NaN,NaN,NaN,0,BBox,1.0,739.0,482.0,1170.0,2048,1536
3399,3514,270_PICT1301_S02.jpg,2146.00 1285.00,Leopard,NaN,1704.00 691.00 2588.00 1879.00,NaN,NaN,NaN,0,BBox,1704.0,691.0,2588.0,1879.0,2592,1944


In [559]:
%%time
df_tiger = determ_width_height(df_tiger, "Тигр")
df_tiger

Wall time: 845 ms


,index,id,center,class,subclass,bbox,convex,headtail,orient,relevance,mark_tool,xmin,ymin,xmax,ymax,width,height
0,0,001_1.jpg,899.50 755.00,Tiger,NaN,148.00 406.00 1651.00 1104.00,NaN,NaN,NaN,0,BBox,148.0,406.0,1651.0,1104.0,2048,1536
1,1,002_758.jpg,1340.00 830.00,Tiger,NaN,730.00 524.00 1950.00 1136.00,NaN,NaN,NaN,0,BBox,730.0,524.0,1950.0,1136.0,2048,1536
2,2,002_76.jpg,2593.50 1531.50,Tiger,NaN,1928.00 850.00 3259.00 2213.00,NaN,NaN,NaN,0,BBox,1928.0,850.0,3259.0,2213.0,3264,2448
3,3,002_761.jpg,324.50 799.00,Tiger,NaN,1.00 582.00 648.00 1016.00,NaN,NaN,NaN,0,BBox,1.0,582.0,648.0,1016.0,2048,1536
4,4,002_762.jpg,570.50 779.50,Tiger,NaN,138.00 564.00 1003.00 995.00,NaN,NaN,NaN,0,BBox,138.0,564.0,1003.0,995.0,2048,1536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,3532,109_IMG_0158_S02.jpg,472.50 765.00,Tiger,NaN,0.00 512.00 945.00 1018.00,NaN,NaN,NaN,0,BBox,0.0,512.0,945.0,1018.0,2048,1536
3455,3533,109_IMG_0157_S02.jpg,599.00 749.00,Tiger,NaN,94.00 495.00 1104.00 1003.00,NaN,NaN,NaN,0,BBox,94.0,495.0,1104.0,1003.0,2048,1536
3456,3534,109_IMG_0154_S02.jpg,916.00 730.00,Tiger,NaN,442.00 493.00 1390.00 967.00,NaN,NaN,NaN,0,BBox,442.0,493.0,1390.0,967.0,2048,1536
3457,3535,109_IMG_0143_S02.jpg,1683.50 705.50,Tiger,NaN,1337.00 530.00 2030.00 881.00,NaN,NaN,NaN,0,BBox,1337.0,530.0,2030.0,881.0,2048,1536


In [560]:
%%time
df_princess = determ_width_height(df_princess, "Принцeссa")
df_princess

Wall time: 96.5 ms


,index,id,center,class,subclass,bbox,convex,headtail,orient,relevance,mark_tool,xmin,ymin,xmax,ymax,width,height
0,0,Princess_387.jpg,2126.50 1212.00,Tiger,NaN,989.00 630.00 3264.00 1794.00,NaN,NaN,NaN,0,BBox,989.00,630.0,3264.00,1794.0,3264,2448
1,1,Princess_388.jpg,802.00 1678.00,Tiger,NaN,0.00 1000.00 1604.00 2356.00,NaN,NaN,NaN,0,BBox,0.00,1000.0,1604.00,2356.0,3264,2448
2,2,Princess_389.jpg,1347.36 1694.50,Tiger,NaN,-3.29 981.00 2698.00 2408.00,NaN,NaN,NaN,0,BBox,-3.29,981.0,2698.00,2408.0,3264,2448
3,3,Princess_39.jpg,387.50 985.00,Tiger,NaN,0.00 758.00 775.00 1212.00,NaN,NaN,NaN,0,BBox,0.00,758.0,775.00,1212.0,2048,1536
4,4,Princess_391.jpg,2479.50 1219.50,Tiger,NaN,1698.00 640.00 3261.00 1799.00,NaN,NaN,NaN,0,BBox,1698.00,640.0,3261.00,1799.0,3264,2448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,356,Princess_107.jpg,2532.00 1315.50,Tiger,NaN,1800.00 456.00 3264.00 2175.00,NaN,NaN,NaN,0,BBox,1800.00,456.0,3264.00,2175.0,3264,2448
355,357,Princess_1.jpg,740.29 500.00,Tiger,NaN,206.00 156.00 1274.58 844.00,NaN,NaN,NaN,0,BBox,206.00,156.0,1274.58,844.0,1273,955
356,358,Princess_240.jpg,2830.00 1536.00,Tiger,NaN,2396.00 725.00 3264.00 2347.00,NaN,NaN,NaN,0,BBox,2396.00,725.0,3264.00,2347.0,3264,2448
357,359,Princess_99.jpg,2063.50 1163.00,Tiger,NaN,863.00 621.00 3264.00 1705.00,NaN,NaN,NaN,0,BBox,863.00,621.0,3264.00,1705.0,3264,2448


In [561]:
def new_df(df):
    df_new = pd.DataFrame()
    df_new['filename'] = df['id']
    df_new['width'] = df['width']
    df_new['height'] = df['height']
    df_new['class'] = df['class']
    df_new['xmin'] = df['xmin']
    df_new['ymin'] = df['ymin']
    df_new['xmax'] = df['xmax']
    df_new['ymax'] = df['ymax']
    return df_new

In [562]:
leopard = new_df(df_leopard)
leopard

,filename,width,height,class,xmin,ymin,xmax,ymax
0,01020010_S02.jpg,3264,2448,Leopard,1218.0,1000.0,2481.0,1635.0
1,212_IMG_0841_S02.jpg,2048,1536,Leopard,554.0,730.0,1670.0,1506.0
2,212_IMG_0840_S02.jpg,2048,1536,Leopard,997.0,724.0,2008.0,1394.0
3,212_IMG_0839_S02.jpg,2048,1536,Leopard,1424.0,734.0,2045.0,1363.0
4,212_IMG_0838_S02.jpg,2048,1536,Leopard,1824.0,763.0,2045.0,1293.0
...,...,...,...,...,...,...,...,...
3396,216_08210217_S02.jpg,2048,1536,Leopard,568.0,756.0,1619.0,1365.0
3397,236_IMG_3658_S02.jpg,2048,1536,Leopard,406.0,436.0,2024.0,1493.0
3398,246_IMG_1651_S02.jpg,2048,1536,Leopard,1.0,739.0,482.0,1170.0
3399,270_PICT1301_S02.jpg,2592,1944,Leopard,1704.0,691.0,2588.0,1879.0


In [563]:
tiger = new_df(df_tiger)
tiger

,filename,width,height,class,xmin,ymin,xmax,ymax
0,001_1.jpg,2048,1536,Tiger,148.0,406.0,1651.0,1104.0
1,002_758.jpg,2048,1536,Tiger,730.0,524.0,1950.0,1136.0
2,002_76.jpg,3264,2448,Tiger,1928.0,850.0,3259.0,2213.0
3,002_761.jpg,2048,1536,Tiger,1.0,582.0,648.0,1016.0
4,002_762.jpg,2048,1536,Tiger,138.0,564.0,1003.0,995.0
...,...,...,...,...,...,...,...,...
3454,109_IMG_0158_S02.jpg,2048,1536,Tiger,0.0,512.0,945.0,1018.0
3455,109_IMG_0157_S02.jpg,2048,1536,Tiger,94.0,495.0,1104.0,1003.0
3456,109_IMG_0154_S02.jpg,2048,1536,Tiger,442.0,493.0,1390.0,967.0
3457,109_IMG_0143_S02.jpg,2048,1536,Tiger,1337.0,530.0,2030.0,881.0


In [570]:
princess = new_df(df_princess)
princess['class'] = 'Princess'
princess

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Princess_387.jpg,3264,2448,Princess,989.00,630.0,3264.00,1794.0
1,Princess_388.jpg,3264,2448,Princess,0.00,1000.0,1604.00,2356.0
2,Princess_389.jpg,3264,2448,Princess,-3.29,981.0,2698.00,2408.0
3,Princess_39.jpg,2048,1536,Princess,0.00,758.0,775.00,1212.0
4,Princess_391.jpg,3264,2448,Princess,1698.00,640.0,3261.00,1799.0
...,...,...,...,...,...,...,...,...
354,Princess_107.jpg,3264,2448,Princess,1800.00,456.0,3264.00,2175.0
355,Princess_1.jpg,1273,955,Princess,206.00,156.0,1274.58,844.0
356,Princess_240.jpg,3264,2448,Princess,2396.00,725.0,3264.00,2347.0
357,Princess_99.jpg,3264,2448,Princess,863.00,621.0,3264.00,1705.0


In [571]:
leopard.to_csv('leopard.csv', index = None)
princess.to_csv('princess.csv', index = None)
tiger.to_csv('tiger.csv', index = None)

In [572]:
t_leopard = pd.read_csv('leopard.csv')
t_princess = pd.read_csv('princess.csv')
t_tiger = pd.read_csv('tiger.csv')

In [573]:
t_leopard

,filename,width,height,class,xmin,ymin,xmax,ymax
0,01020010_S02.jpg,3264,2448,Leopard,1218.0,1000.0,2481.0,1635.0
1,212_IMG_0841_S02.jpg,2048,1536,Leopard,554.0,730.0,1670.0,1506.0
2,212_IMG_0840_S02.jpg,2048,1536,Leopard,997.0,724.0,2008.0,1394.0
3,212_IMG_0839_S02.jpg,2048,1536,Leopard,1424.0,734.0,2045.0,1363.0
4,212_IMG_0838_S02.jpg,2048,1536,Leopard,1824.0,763.0,2045.0,1293.0
...,...,...,...,...,...,...,...,...
3396,216_08210217_S02.jpg,2048,1536,Leopard,568.0,756.0,1619.0,1365.0
3397,236_IMG_3658_S02.jpg,2048,1536,Leopard,406.0,436.0,2024.0,1493.0
3398,246_IMG_1651_S02.jpg,2048,1536,Leopard,1.0,739.0,482.0,1170.0
3399,270_PICT1301_S02.jpg,2592,1944,Leopard,1704.0,691.0,2588.0,1879.0


In [574]:
t_princess

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Princess_387.jpg,3264,2448,Princess,989.00,630.0,3264.00,1794.0
1,Princess_388.jpg,3264,2448,Princess,0.00,1000.0,1604.00,2356.0
2,Princess_389.jpg,3264,2448,Princess,-3.29,981.0,2698.00,2408.0
3,Princess_39.jpg,2048,1536,Princess,0.00,758.0,775.00,1212.0
4,Princess_391.jpg,3264,2448,Princess,1698.00,640.0,3261.00,1799.0
...,...,...,...,...,...,...,...,...
354,Princess_107.jpg,3264,2448,Princess,1800.00,456.0,3264.00,2175.0
355,Princess_1.jpg,1273,955,Princess,206.00,156.0,1274.58,844.0
356,Princess_240.jpg,3264,2448,Princess,2396.00,725.0,3264.00,2347.0
357,Princess_99.jpg,3264,2448,Princess,863.00,621.0,3264.00,1705.0


In [575]:
t_tiger

,filename,width,height,class,xmin,ymin,xmax,ymax
0,001_1.jpg,2048,1536,Tiger,148.0,406.0,1651.0,1104.0
1,002_758.jpg,2048,1536,Tiger,730.0,524.0,1950.0,1136.0
2,002_76.jpg,3264,2448,Tiger,1928.0,850.0,3259.0,2213.0
3,002_761.jpg,2048,1536,Tiger,1.0,582.0,648.0,1016.0
4,002_762.jpg,2048,1536,Tiger,138.0,564.0,1003.0,995.0
...,...,...,...,...,...,...,...,...
3454,109_IMG_0158_S02.jpg,2048,1536,Tiger,0.0,512.0,945.0,1018.0
3455,109_IMG_0157_S02.jpg,2048,1536,Tiger,94.0,495.0,1104.0,1003.0
3456,109_IMG_0154_S02.jpg,2048,1536,Tiger,442.0,493.0,1390.0,967.0
3457,109_IMG_0143_S02.jpg,2048,1536,Tiger,1337.0,530.0,2030.0,881.0
